In [1]:
import numpy as np
import parser

In [2]:
def nmae(y, y_, minr=1, maxr=5):
    denominator = np.count_nonzero(y.flatten())
    return np.sum(np.absolute(y-y_))/(denominator * (maxr-minr))

In [3]:
def S(W, lambda_val):
    U, S, V = np.linalg.svd(W, full_matrices=False)
    S = np.maximum(S - lambda_val, 0)
    S = np.diag(S)
    return np.dot(U, np.dot(S, V))

In [4]:
def MCAD(Y, lambda_val):
    X = np.random.rand(*Y.shape) * 5
    R = 1 * (Y!=0)
    num_iters = 25
    for i in range(num_iters):
        B = X + Y - np.multiply(R, X)
        U, S, V = np.linalg.svd(B, full_matrices=False)
        objective = (np.linalg.norm(Y - np.multiply(R, X), ord='fro') ** 2) + (lambda_val * (np.sum(S)))
        S_ = np.sign(S) * np.maximum(0, np.abs(S - lambda_val/2))
        X = np.dot(U, np.dot(np.diag(S_), V))
#         print objective
    return X

In [ ]:
def main():
    dd = "ml-100k/"
    movies = parser.load_data(dd + "u.item", parser.Movie, '|')
    ratings = []
    for i in range(1,1+5):
        ratings.append( [parser.load_data(dd + "u" + str(i) + ".base", parser.Rating, '\t'), parser.load_data(dd + "u" + str(i) + ".test", parser.Rating, '\t')] )
    users = parser.load_data(dd + "u.user", parser.User, '|')
    lambda_values = [0.01, 0.1, 1, 10, 100]
    for lambda_val in lambda_values: 
        errors = []
        for fold in ratings:
            #Transpose ratings to convert standard problem algorithm 
            R = parser.R_matrix(len(users), len(movies), fold[0])
            R_test = parser.R_matrix(len(users), len(movies), fold[1])
            R_recon = MCAD(R, lambda_val)
            R_recon = R_recon * (R_test != 0)
            errors.append(nmae(R_test, R_recon))
        print("Lambda",lambda_val,", average NMAE over all folds:", sum(errors)/len(errors)*1.0)


In [ ]:
main()

('Lambda', 0.01, ', average NMAE over all folds:', 0.42714296756898767)
